In [19]:
import pandas as pd
from __future__ import unicode_literals
from hazm import *
from collections import Counter
from tqdm import tqdm

In [42]:
df = pd.read_csv('normalize_divar_dataset.csv')

In [53]:
df.head()

,Unnamed: 0,Unnamed: 0.1,index,cat1,cat2,cat3,title,description,url,sub_title,price,size,year,room_number
0,0,0,0,home-kitchen,appliance,washing-machines,ماشین لباسشویی دوقلوی کودک,سازنده برند قدیمی ناسیونال,https://divar.ir/v/ماشین-لباسشویی-دوقلوی-کودک_...,یک ربع پیش در تهران، جوانمرد قصاب,1850000.0,0,0,0
1,1,1,1,home-kitchen,appliance,washing-machines,لباسشویی بست BOST,لباسشویی کاملا سالم خدارو شکر مشکلی نداره از ه...,https://divar.ir/v/لباسشویی-بست-bost_ماشین-لبا...,نیم ساعت پیش در تهران، آذربایجان,8000000.0,0,0,0
2,2,2,2,home-kitchen,appliance,washing-machines,ماشین لباسشویی سامسونگ,ماشین لباسشویی سامسونگ کره‌ای اصلی سالم تمیز ب...,https://divar.ir/v/ماشین-لباسشویی-سامسونگ_ماشی...,نیم ساعت پیش در تهران، ابن بابویه,5500000.0,0,0,0
3,3,3,3,home-kitchen,appliance,washing-machines,موتور لباسشویی هایر,یک عدد موتور لباسشویی برای همه ماشین لباسشویی‌...,https://divar.ir/v/موتور-لباسشویی-هایر_ماشین-ل...,۱ ساعت پیش در تهران، جنت‌آباد جنوبی,500000.0,0,0,0
4,4,4,4,home-kitchen,appliance,washing-machines,لباسشوئی سامسونگ,ساعات پاسخگویی ۲۰۹ توجه: قیمت کالا مقطوع است و...,https://divar.ir/v/لباسشوئی-سامسونگ_ماشین-لباس...,۱ ساعت پیش در تهران، جنت‌آباد شمالی,12000000.0,0,0,0


In [12]:
df["description"] = df['description'].fillna('توضیحات ندارد')
df["sub_title"] = df['sub_title'].fillna('توضیحات ندارد')

In [13]:

#### split to words
split_it=[]
signs = [",", ".", "?", ";", ":", "(", ")", "$", "%", "!", "\'", "\"", "{", "}", "[", "]", "&", 
           "،", "؛", "«", "»", "؟", "!", " ", "‌","?","@","1"]
for i in range(len(df)):
    
    a = word_tokenize(df.iat[i,6])
    b = word_tokenize(df.iat[i,7])
    c = word_tokenize(df.iat[i,9])
    
    for j in a:   
        if j in signs :
            continue
        else:
            split_it.append(j)
    for j in b:   
        if j in signs :
            continue
        else:
            split_it.append(j)
            
    for j in c:   
        if j in signs :
            continue
        else:
            split_it.append(j)            
                
            
            
        

In [15]:
len(split_it)

61379

In [16]:
#### find unique words

wordset=[]
for i in split_it:
    if i in wordset  :
        continue
    else : 
        wordset.append(i)
print(len(wordset))    

7300


In [21]:
def calculateBOW(wordset,l_doc):
  tf_diz = dict.fromkeys(wordset,0)
  for word in l_doc:
        tf_diz[word]=l_doc.count(word)
  return tf_diz

In [22]:
df_bow = pd.DataFrame()
for i in tqdm(range(len(df))):
    a = word_tokenize(df.iat[i,6])
    b = word_tokenize(df.iat[i,7])
    c = word_tokenize(df.iat[i,9])
    bow1 = calculateBOW(wordset,a+b+c)
    bow1=pd.DataFrame.from_dict([bow1])
    df_bow = df_bow.append(bow1, ignore_index = True)
    df_bow.head()

100%|██████████| 1017/1017 [08:24<00:00,  2.01it/s]


In [40]:
df_bow =  df_bow.dropna(axis=1)

In [49]:
extracted_col  = df[['cat1','cat2','cat3','price','size','year','room_number']]


In [50]:
df2 = df_bow.join(extracted_col)

In [52]:
df2.head()

,ماشین,لباسشویی,دوقلوی,کودک,سازنده,برند,قدیمی,ناسیونال,یک,ربع,...,فرمون,دوخته,فرزانگان,cat1,cat2,cat3,price,size,year,room_number
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,home-kitchen,appliance,washing-machines,1850000.0,0,0,0
1,0,3,0,0,0,0,0,0,0,0,...,0,0,0,home-kitchen,appliance,washing-machines,8000000.0,0,0,0
2,3,2,0,0,0,0,0,0,0,0,...,0,0,0,home-kitchen,appliance,washing-machines,5500000.0,0,0,0
3,1,2,0,0,0,0,0,0,1,0,...,0,0,0,home-kitchen,appliance,washing-machines,500000.0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,home-kitchen,appliance,washing-machines,12000000.0,0,0,0


In [55]:
df2.to_csv('N_gram.csv')